In [ ]:
# 最終課題コンペ用 Fine-tuning テンプレート（unsloth）- 改変版
# --- 本コードは講義で配布されたサンプルプログラムを改修したものです
# --- サンプルコード自体の著作権は 講義主催である東京大学に属します

In [ ]:
# ------------------------------------------------------------------------------------
# パッケージをインストール
!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -qU

!pip install --upgrade torch torchvision torchaudio -qU
!pip install --upgrade xformers -qU

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3" -qU

In [ ]:
# ------------------------------------------------------------------------------------
# パラメータ設定
device = "cuda" if torch.cuda.is_available() else "cpu"

# データを下記に配置してください
input_path  = "/content/input/ichikara-instruction-003-001-1.json"
eval_path   = "/content/eval/elyza-tasks-100-TV_0.jsonl"
result_path = "/content/results"

# ------------------------------------------------------------------------------------
# 学習パラメータを設定
model_id = "llm-jp/llm-jp-3-13b"
new_model_id = "llm-jp-3-13b-it"      # Fine-Tuningしたモデルにつけたい名前
dtype = None                          # Noneにしておけば自動で設定                                                   +
load_in_4bit = True                   # 今回は13Bモデルを扱うためTrue
max_seq_length = 512                  # unslothではRoPEをサポートしているのでコンテキスト長は自由に設定可能

# パラメータをPack
config={
    "model_id": model_id,
    "learning_rate": 2e-5,
    "per_device_train_batch_size": 4,
    "gradient_accumulation_steps": 4,
    "num_train_epochs":3,
    "warmup_steps": 10,
    "max_steps": -1,
    "lora_r": 32,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "lora_bias": "none",
    "lora_use_rslora": False,
    "lora_loftq_config": None,
    "model_max_seq_length": max_seq_length,
    "model_dtype": dtype,
    "model_load_in_4bit": load_in_4bit,
    "seed": 3407,
    "max_seq_length": max_seq_length
}

In [ ]:
# ------------------------------------------------------------------------------------
# llm-jp/llm-jp-3-13bを4bit量子化のqLoRA設定でロード。
from unsloth import FastLanguageModel
model_A, tokenizer = FastLanguageModel.from_pretrained(    # FastLanguageModel インスタンスを作成
    model_name   = config.model_id,
    dtype        = config.model_dtype,
    load_in_4bit = config.model_load_in_4bit,
    trust_remote_code= True,
)

model = FastLanguageModel.get_peft_model(                  # SFT用のモデルを用意
    model_A,
    r              = config.lora_r,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha     = config.lora_alpha,
    lora_dropout   = config.lora_dropout,
    bias           = config.lora_bias,
    random_state   = config.seed,
    use_rslora     = config.lora_use_rslora,
    loftq_config   = config.lora_loftq_config,
    max_seq_length = config.max_seq_length,
    use_gradient_checkpointing = "unsloth",
)

In [ ]:
# ------------------------------------------------------------------------------------
# データセットをトレーニングとテストデータセットに分割してロード
from datasets import load_dataset
train_dataset = load_dataset("json", data_files= input_path, split="train[:80%]" )
test_dataset = load_dataset("json", data_files= input_path, split="train[80%:]")

#  各データをプロンプトに合わせた形式に合わせる関数「formatting_prompts_func」
EOS_TOKEN = tokenizer.eos_token                  # トークナイザーのEOSトークン（文末トークン）
prompt = f"""### 指示\n{input}\n### 回答\n"""      # 学習時のプロンプトフォーマットの定義

def formatting_prompts_func(examples):
    input = examples["text"]                          # 入力データ
    output = examples["output"]                       # 出力データ
    text = prompt.format(input, output) + EOS_TOKEN   # プロンプトの作成
    return { "formatted_text" : text, }               # 新しいフィールド "formatted_text" を返す

# 両データセットにPromptフォーマットを適用
train_dataset = train_dataset.map( formatting_prompts_func, num_proc= 4 )
test_dataset = test_dataset.map( formatting_prompts_func, num_proc= 4 )


In [ ]:
# ------------------------------------------------------------------------------------
# ファインチューニング開始
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from unsloth import is_bfloat16_supported

# EarlyStoppingCallbackの設定
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience  = 3,         # 改善がない場合に許容するエポック数
    early_stopping_threshold = 0.0        # 改善があったとみなす最小の変化量
)

# Trainerを作成
trainer = SFTTrainer(
    model              = model,
    tokenizer          = tokenizer,
    train_dataset      = train_dataset,
    eval_dataset       = test_dataset,
    max_seq_length     = config.max_seq_length,
    dataset_text_field = "formatted_text",
    packing            = False,
    callbacks=[early_stopping_callback],  # コールバックへEarlyStoppingを設定
    args = TrainingArguments(
        per_device_train_batch_size = config.per_device_train_batch_size,
        gradient_accumulation_steps = config.gradient_accumulation_steps,
        num_train_epochs = config.num_train_epochs,
        warmup_steps     = config.warmup_steps,
        max_steps        = config.max_steps,
        learning_rate    = config.learning_rate,
        seed             = config.seed,
        evaluation_strategy = "steps",       # ステップ単位で評価
        eval_steps       = 20,               # 20ステップごとに評価
        save_strategy    = "steps",          # ステップ単位でモデルを保存
        save_steps       = 60,               # 60ステップごとに保存
        save_total_limit = 3,
        load_best_model_at_end = True,        # ベストモデルをロード
        metric_for_best_model  = "eval_loss", # EarlyStoppingの基準メトリクス/損失を基準に最良モデルを選択
        greater_is_better      = False,       # eval_lossは小さい方が良い
        output_dir       = "outputs",
        report_to        = "wandb",
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        group_by_length  = True,
        logging_steps    = 10,
    ),
)

In [ ]:
# ------------------------------------------------------------------------------------
# 学習実行
trainer_stats = trainer.train()


In [ ]:
# ------------------------------------------------------------------------------------
# Elyza-100をDatasetとして読み込む
import json
eval_datasets = []
elyza_tasks_path = eval_path
with open(elyza_tasks_path, "r") as f:
    item = ""
    for line in f:
      line = line.strip()
      item += line
      if item.endswith("}"):
        eval_datasets.append(json.loads(item))
        item = ""


In [ ]:
# ファインチューニングしたモデルを用いて、Elyza-100質問タスクを実行
from tqdm import tqdm
FastLanguageModel.for_inference(model)
model.eval()
results = []
for dt in tqdm(eval_datasets):
  input = dt["input"]
  prompt = f"""### 指示\n{input}\n### 回答\n"""
  inputs = tokenizer([prompt], return_tensors = "pt")
  outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True, do_sample=False, repetition_penalty=1.2)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split('\n### 回答')[-1]
  results.append({"task_id": dt["task_id"], "input": input, "output": prediction})

In [ ]:
# ------------------------------------------------------------------------------------
# jsonlで保存
jsonl_result_path = result_path + f"/{new_model_id}_output.jsonl",
with open( jsonl_result_path, 'w', encoding='utf-8') as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')


In [ ]:
# LoRAアダプタだけをH保存
HF_TOKEN = "huggingface Access Token"
model.push_to_hub_merged(
    new_model_id+"_lora",
    tokenizer=tokenizer,
    save_method="lora",
    token=HF_TOKEN,
    private=False
)